# Decision Trees

What need to be decided on ? <br>
- split feature <br>
- split point <br>
- when to stop splitting ? <br>

Training : <br>
- Calculate unformation gain with each possible split <br>
- Divide set with that feature and value that gives the most IG (information gain = entropy of the parent - entropy of children * weighted average) <br>
- Divide tree and do the same for all created branches ... <br>
- ... until a stopping criteria is reached <br>
stopping criteria : maximum depth, minimum number of samples, min impurity decrease <br>

Testing : <br>
- follow the tree until we reach a leaf node <br>
- return the most common class label 

Entropy : $E = - \displaystyle \sum p(X) . log_{2}(p(X)) \space $  where  $ \space p(X) = \frac{\text{card(X)}}{n}$

In [2]:
import numpy as np
from collections import Counter

In [34]:
class Node: 
    def __init__(self, feature = None, threshold = None, left= None, right = None, *, value=None):
        self.feature = feature 
        self.threshold = threshold
        self.left = left
        self.right = right 
        self.value = None

    def is_leaf_node(self):
        return self.value is not None

class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None 

    def fit(self,X,y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X,y)

    def _grow_tree(self,X,y,depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        #check the stopping criteria 
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split 
        best_feature, best_thresh  = self._best_split(X,y,feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:,best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)

    def _most_common_label(self,y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value 

    def _best_split(self,X,y,feat_idxs):
        best_gain = -1
        split_idx, split_treshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:,feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                #calculate the information gain 
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain: 
                    best_gain = gain 
                    split_idx = feat_idx 
                    split_threshold = thr 

        return split_idx, split_threshold
    

    def _information_gain(self, y, X_column, threshold):
        # entropy of parent - weighted average * entropy of children 
        #parent entropy 
        parent_entropy = self._entropy(y)

        #create children 
        left_idx, right_idx = self._split(X_column, threshold) 
        if len(left_idx)==0 or len(right_idx) == 0: 
            return 0
        
        #children entropy 
        n = len(y)
        n_l, n_r = len(left_idx), len(right_idx)
        e_l, e_r = self._entropy(y[left_idx]), self._entropy(y[right_idx])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r 
        
        # calculate the information gain 
        information_gain = parent_entropy - child_entropy 
        return information_gain


    def _split(self,X_column, split_tresh):
        left_idxs = np.argwhere(X_column<=split_tresh).flatten()
        right_idxs = np.argwhere(X_column>split_tresh).flatten()
        return left_idxs, right_idxs


    def _entropy(self,y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return - np.sum([p * np.log(p) for p in ps if p>0])


    
    def predict(self, X):
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    def _traverse_tree(self,x,node):
        if node.is_leaf_node():
            return node.value()
        
        if x[node.feature] <= node.threshold & np.isnan(node.threshold)==False :
            return self._traverse_tree(x,node.left)
        return self._traverse_tree(x,node.right)


In [35]:
from sklearn import datasets 
from sklearn.model_selection import train_test_split 


In [36]:
data = datasets.load_breast_cancer()
X,y = data.data, data.target 

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state= 1234)

clf = DecisionTree()
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)

def accuracy(y_test, y_pred):
    return print(np.sum(y_test==y_pred) / len(y_test))

accuracy(y_test, predictions)

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''